В цьому домашньому завданні ми знову працюємо з даними з нашого змагання ["Bank Customer Churn Prediction (DLU Course)"](https://www.kaggle.com/t/7c080c5d8ec64364a93cf4e8f880b6a0).

Тут ми побудуємо рішення задачі класифікації з використанням алгоритмів бустингу: XGBoost та LightGBM, а також використаємо бібліотеку HyperOpt для оптимізації гіперпараметрів.

0. Зчитайте дані `train.csv` в змінну `raw_df` та скористайтесь наведеним кодом нижче аби розділити дані на трнувальні та валідаційні і розділити дані на ознаки з матириці Х та цільову змінну. Назви змінних `train_inputs, train_targets, train_inputs, train_targets` можна змінити на ті, які Вам зручно.

  Наведений скрипт - частина отриманого мною скрипта для обробки даних. Ми тут не викнуємо масштабування та обробку категоріальних змінних, бо хочемо це делегувати алгоритмам, які будемо використовувати. Якщо щось не розумієте в наведених скриптах, рекомендую розібратись: навичка читати код - важлива складова роботи в машинному навчанні.

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from numpy.f2py.crackfortran import verbose
from sklearn.model_selection import train_test_split
from typing import Tuple, Dict, Any


def split_train_val(df: pd.DataFrame, target_col: str, test_size: float = 0.2, random_state: int = 42) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Split the dataframe into training and validation sets.

    Args:
        df (pd.DataFrame): The raw dataframe.
        target_col (str): The target column for stratification.
        test_size (float): The proportion of the dataset to include in the validation split.
        random_state (int): Random state for reproducibility.

    Returns:
        Tuple[pd.DataFrame, pd.DataFrame]: Training and validation dataframes.
    """
    train_df, val_df = train_test_split(df, test_size=test_size, random_state=random_state, stratify=df[target_col])
    return train_df, val_df


def separate_inputs_targets(df: pd.DataFrame, input_cols: list, target_col: str) -> Tuple[pd.DataFrame, pd.Series]:
    """
    Separate inputs and targets from the dataframe.

    Args:
        df (pd.DataFrame): The dataframe.
        input_cols (list): List of input columns.
        target_col (str): Target column.

    Returns:
        Tuple[pd.DataFrame, pd.Series]: DataFrame of inputs and Series of targets.
    """
    inputs = df[input_cols].copy()
    targets = df[target_col].copy()
    return inputs, targets

In [3]:
raw_df = pd.read_csv('../../../../bank-customer-churn-prediction-dlu/data/train.csv', index_col=0)

train_df, val_df = split_train_val(raw_df, target_col='Exited')

train_inputs, train_targets = separate_inputs_targets(train_df, input_cols=raw_df.columns.drop('Exited').to_list(), target_col='Exited')
val_inputs, val_targets = separate_inputs_targets(val_df, input_cols=raw_df.columns.drop('Exited').to_list(), target_col='Exited')

In [4]:
train_inputs.dtypes

CustomerId         float64
Surname             object
CreditScore        float64
Geography           object
Gender              object
Age                float64
Tenure             float64
Balance            float64
NumOfProducts      float64
HasCrCard          float64
IsActiveMember     float64
EstimatedSalary    float64
dtype: object

1. В тренувальному та валідаційному наборі перетворіть категоріальні ознаки на тип `category`. Можна це зробити двома способами:
 1. `df[col_name].astype('category')`, як було продемонстровано в лекції
 2. використовуючи метод `pd.Categorical(df[col_name])`

In [5]:
for col in train_inputs.select_dtypes(include='object').columns:
    train_inputs[col] = pd.Categorical(train_inputs[col])
    val_inputs[col] = pd.Categorical(val_inputs[col])

In [6]:
train_inputs.dtypes

CustomerId          float64
Surname            category
CreditScore         float64
Geography          category
Gender             category
Age                 float64
Tenure              float64
Balance             float64
NumOfProducts       float64
HasCrCard           float64
IsActiveMember      float64
EstimatedSalary     float64
dtype: object

2. Навчіть на отриманих даних модель `XGBoostClassifier`. Параметри алгоритму встановіть на свій розсуд, ми далі будемо їх тюнити. Рекомендую тренувати не дуже складну модель.

  Опис всіх конфігураційних параметрів XGBoostClassifier - тут https://xgboost.readthedocs.io/en/stable/parameter.html#global-config

  **Важливо:** зробіть такі налаштування `XGBoostClassifier` аби він самостійно обробляв незаповнені значення в даних і обробляв категоріальні колонки.

  Можна також, якщо працюєте в Google Colab, увімкнути можливість використання GPU (`Runtime -> Change runtime type -> T4 GPU`) і встановити параметр `device='cuda'` в `XGBoostClassifier` для пришвидшення тренування бустинг моделі.
  
  Після тренування моделі
  1. Виміряйте точність з допомогою AUROC на тренувальному та валідаційному наборах.
  2. Зробіть висновок про отриману модель: вона хороша/погана, чи є high bias/high variance?
  3. Порівняйте якість цієї моделі з тою, що ви отрмали з використанням DecisionTrees раніше. Чи вийшло покращити якість?

In [7]:
# Verify that missing values are handled correctly
train_inputs.isnull().sum()

CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
dtype: int64

In [8]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, classification_report

xgb_clf = XGBClassifier(
    booster='gbtree',  # default value. Make it explicit
    max_depth=5,
    n_estimators=50,
    enable_categorical=True,  # handle categirical features
    eval_metric='auc',
    objective='binary:logistic',
)

xgb_clf.fit(train_inputs, train_targets)

train_pred = xgb_clf.predict(train_inputs)
val_pred = xgb_clf.predict(val_inputs)

print(classification_report(train_targets, train_pred, digits=4))
print(classification_report(val_targets, val_pred, digits=4))

              precision    recall  f1-score   support

         0.0     0.9651    0.9865    0.9757      9558
         1.0     0.9422    0.8604    0.8994      2442

    accuracy                         0.9608     12000
   macro avg     0.9536    0.9234    0.9375     12000
weighted avg     0.9604    0.9608    0.9602     12000

              precision    recall  f1-score   support

         0.0     0.9255    0.9460    0.9357      2390
         1.0     0.7684    0.7016    0.7335       610

    accuracy                         0.8963      3000
   macro avg     0.8470    0.8238    0.8346      3000
weighted avg     0.8936    0.8963    0.8945      3000



In [9]:
train_auc = roc_auc_score(train_targets, xgb_clf.predict_proba(train_inputs)[:, 1])
val_auc = roc_auc_score(val_targets, xgb_clf.predict_proba(val_inputs)[:, 1])

print(f"Train AUROC: {train_auc:.4f}")
print(f"Validation AUROC: {val_auc:.4f}")

Train AUROC: 0.9877
Validation AUROC: 0.9289


**Висновок:**
Дана модель є поганою, бо вона перенавчана в даному випадку. Вона добре себе показує на тренувальних даних, проте на валідаційних - погано. Тут є high variance в даному випадку. У порівнянні із Модулем 2.3 де ми вчили DecisionTreeClassifier, якість моделі погіршилась, хоча ми виставили також параметр `max_depth=5` для DecisionTreeClassifier. Також із лекції було сказано те, що треба робити глубину менше, щоб робити їх більш простими і такими "пеньками" проте для експеременту залишу поки глубину в 5.

3. Використовуючи бібліотеку `Hyperopt` і приклад пошуку гіперпараметрів для `XGBoostClassifier` з лекції знайдіть оптимальні значення гіперпараметрів `XGBoostClassifier` для нашої задачі. Задайте свою сітку гіперпараметрів виходячи з тих параметрів, які ви б хотіли перебрати. Поставте кількість раундів в підборі гіперпараметрів рівну **20**.

  **Увага!** Для того, аби скористатись hyperopt, нам треба задати функцію `objective`. В ній ми маємо задати loss - це може будь-яка метрика, але бажано використовувтаи ту, яка цільова в вашій задачі. Чим менший лосс - тим ліпша модель на думку hyperopt. Тож, тут нам треба задати loss - негативне значення AUROC. В лекції ми натомість використовували Accuracy.

  Після успішного завершення пошуку оптимальних гіперпараметрів
    - виведіть найкращі значення гіперпараметрів
    - створіть в окремій зміній `final_clf` модель `XGBoostClassifier` з найкращими гіперпараметрами
    - навчіть модель `final_clf`
    - оцініть якість моделі `final_clf` на тренувальній і валідаційній вибірках з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи стала вона краще порівняно з попереднім пунктом (2) цього завдання?

In [10]:
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK

def objective(params: Dict[str, Any]) -> Dict[str, Any]:
    xgb_clf = XGBClassifier(
        booster='gbtree',
        max_depth=params['max_depth'],
        n_estimators=params['n_estimators'],
        learning_rate=params['learning_rate'],
        enable_categorical=True,
        eval_metric='auc',
        objective='binary:logistic',
    )
    xgb_clf.fit(train_inputs, train_targets)
    val_auc = roc_auc_score(val_targets, xgb_clf.predict_proba(val_inputs)[:, 1])
    loss = -val_auc  # negative AUROC
    return {'loss': loss, 'status': STATUS_OK}

space = {
    'max_depth': hp.choice('max_depth', range(1, 10)),
    'n_estimators': hp.choice('n_estimators', range(5, 600)),
    'learning_rate': hp.uniform('learning_rate', 0.001, 0.3),
}

trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=20,
            trials=trials)

print(best)

100%|██████████| 20/20 [00:11<00:00,  1.68trial/s, best loss: -0.9333520817614377]
{'learning_rate': np.float64(0.055858026642090115), 'max_depth': np.int64(2), 'n_estimators': np.int64(289)}


In [11]:
final_clf = XGBClassifier(
    booster='gbtree',
    max_depth=best['max_depth'],
    n_estimators=best['n_estimators'],
    learning_rate=best['learning_rate'],
    enable_categorical=True,
    eval_metric='auc',
    objective='binary:logistic',
)

final_clf.fit(train_inputs, train_targets)

train_auc = roc_auc_score(train_targets, final_clf.predict_proba(train_inputs)[:, 1])
val_auc = roc_auc_score(val_targets, final_clf.predict_proba(val_inputs)[:, 1])

print(f"Train AUROC: {train_auc:.4f}")
print(f"Validation AUROC: {val_auc:.4f}")

Train AUROC: 0.9566
Validation AUROC: 0.9318


**Висновок:**

Якщо ми порівнюємо із попередніми значеенями, а вони були наступні:

"""
Train AUROC: 0.9877
Validation AUROC: 0.9289
"""

То ми бачимо, що трошки вдалось покращити якісь модель. Хоча вона все ще є перенавчена, бо є різниця між train & validation даними, проте вже не настільки як попередній раз. Також бачимо, що validation auroc покращився із 0.9289 до 0.9314. Для покращення цієї моделі треба ще спробувати поексперементувати із параметрами, а також спробувати max_evals=100 як вказано в туторіалах по hyperopt.

In [12]:
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK

def objective(params: Dict[str, Any]) -> Dict[str, Any]:
    xgb_clf = XGBClassifier(
        booster='gbtree',
        n_estimators=int(params['n_estimators']),
        learning_rate=params['learning_rate'],
        max_depth=int(params['max_depth']),
        min_child_weight=params['min_child_weight'],  # Мінімальна сума ваг всіх вибірок, необхідна в кінцевому вузлі
        subsample=params['subsample'],  # Частка вибірок, що використовуються для побудови кожного дерева
        colsample_bytree=params['colsample_bytree'],  # Частка ознак, що використовуються при побудові кожного дерева
        gamma=params['gamma'],  # Мінімальне зменшення втрат, необхідне для виконання поділу
        reg_alpha=params['reg_alpha'],  # Параметр регуляризації L1 (Lasso)
        reg_lambda=params['reg_lambda'],  # Параметр регуляризації L2 (Ridge)
        enable_categorical=True,
        eval_metric='auc',
        objective='binary:logistic',
        early_stopping_rounds=10,
    )
    xgb_clf.fit(train_inputs, train_targets, eval_set=[(val_inputs, val_targets)], verbose=False)
    val_auc = roc_auc_score(val_targets, xgb_clf.predict_proba(val_inputs)[:, 1])
    loss = -val_auc  # negative AUROC
    return {'loss': loss, 'status': STATUS_OK}

space = {
    'n_estimators': hp.quniform('n_estimators', 50, 700, 25),
    'learning_rate': hp.uniform('learning_rate', 0.001, 0.3),
    'max_depth': hp.quniform('max_depth', 2, 15, 1),
    'min_child_weight': hp.quniform('min_child_weight', 1, 10, 1),
    'subsample': hp.uniform('subsample', 0.5, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0),
    'gamma': hp.uniform('gamma', 0, 0.5),
    'reg_alpha': hp.uniform('reg_alpha', 0, 1),
    'reg_lambda': hp.uniform('reg_lambda', 0, 1)
}

trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=100,
            trials=trials)

print(best)

100%|██████████| 100/100 [00:11<00:00,  8.47trial/s, best loss: -0.9382659990397149]
{'colsample_bytree': np.float64(0.8040919159687082), 'gamma': np.float64(0.06873342159910772), 'learning_rate': np.float64(0.010776187662464413), 'max_depth': np.float64(7.0), 'min_child_weight': np.float64(9.0), 'n_estimators': np.float64(550.0), 'reg_alpha': np.float64(0.7261772522791702), 'reg_lambda': np.float64(0.8845788011207639), 'subsample': np.float64(0.9929225807497607)}


In [13]:
final_clf = XGBClassifier(
    booster='gbtree',
    n_estimators=int(best['n_estimators']),
    learning_rate=best['learning_rate'],
    max_depth=int(best['max_depth']),
    min_child_weight=best['min_child_weight'],  # Мінімальна сума ваг всіх вибірок, необхідна в кінцевому вузлі
    subsample=best['subsample'],  # Частка вибірок, що використовуються для побудови кожного дерева
    colsample_bytree=best['colsample_bytree'],  # Частка ознак, що використовуються при побудові кожного дерева
    gamma=best['gamma'],  # Мінімальне зменшення втрат, необхідне для виконання поділу
    reg_alpha=best['reg_alpha'],  # Параметр регуляризації L1 (Lasso)
    reg_lambda=best['reg_lambda'],  # Параметр регуляризації L2 (Ridge)
    enable_categorical=True,
    eval_metric='auc',
    objective='binary:logistic',
)

final_clf.fit(train_inputs, train_targets)

train_auc = roc_auc_score(train_targets, final_clf.predict_proba(train_inputs)[:, 1])
val_auc = roc_auc_score(val_targets, final_clf.predict_proba(val_inputs)[:, 1])

print(f"Train AUROC: {train_auc:.4f}")
print(f"Validation AUROC: {val_auc:.4f}")

Train AUROC: 0.9626
Validation AUROC: 0.9347


4. Навчіть на наших даних модель LightGBM. Параметри алгоритму встановіть на свій розсуд, ми далі будемо їх тюнити. Рекомендую тренувати не дуже складну модель.

  Опис всіх конфігураційних параметрів LightGBM - тут https://lightgbm.readthedocs.io/en/latest/Parameters.html

  **Важливо:** зробіть такі налаштування LightGBM аби він самостійно обробляв незаповнені значення в даних і обробляв категоріальні колонки.

  Аби передати категоріальні колонки в LightGBM - необхідно виявити їх індекси і передати в параметрі `cat_feature=cat_feature_indexes`

  Після тренування моделі
  1. Виміряйте точність з допомогою AUROC на тренувальному та валідаційному наборах.
  2. Зробіть висновок про отриману модель: вона хороша/погана, чи є high bias/high variance?
  3. Порівняйте якість цієї моделі з тою, що ви отрмали з використанням XGBoostClassifier раніше. Чи вийшло покращити якість?

In [14]:
from lightgbm import LGBMClassifier

cat_feature_indexes = [train_inputs.columns.get_loc(col) for col in train_inputs.select_dtypes(include='category').columns]

lgb_clf = LGBMClassifier(
    max_depth=3,
    n_estimators=2500,
    learning_rate=0.01,
    cat_feature=cat_feature_indexes,
    early_stopping_rounds=50,
)

lgb_clf.fit(train_inputs, train_targets, eval_set=[(val_inputs, val_targets)])

train_pred = lgb_clf.predict(train_inputs)
val_pred = lgb_clf.predict(val_inputs)

print(classification_report(train_targets, train_pred, digits=4))
print(classification_report(val_targets, val_pred, digits=4))


train_auc = roc_auc_score(train_targets, lgb_clf.predict_proba(train_inputs)[:, 1])
val_auc = roc_auc_score(val_targets, lgb_clf.predict_proba(val_inputs)[:, 1])

print(f"Train AUROC: {train_auc:.4f}")
print(f"Validation AUROC: {val_auc:.4f}")

[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Warning] categorical_feature is set with cat_feature=1,3,4, categorical_column=1,3,4 will be ignored. Current value: categorical_feature=1,3,4
[LightGBM] [Info] Number of positive: 2442, number of negative: 9558
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000953 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1571
[LightGBM] [Info] Number of data points in the train set: 12000, number of used features: 12
[LightGBM] [Warning] early_stopping_round is set=50, early

**Висновок:**
Ця модель показує все ж краще ніж попередня без визначення гіперпараметрів. Тут модель все ще перенавчена. auroc для train - 0.9597, в той час коли validation auroc - 0.9359. Це означає, що модель тут high variance. Як покращення, то можна пробувати експерементувати із гіперпараметрами.

5. Використовуючи бібліотеку `Hyperopt` і приклад пошуку гіперпараметрів для `LightGBM` з лекції знайдіть оптимальні значення гіперпараметрів `LightGBM` для нашої задачі. Задайте свою сітку гіперпараметрів виходячи з тих параметрів, які ви б хотіли перебрати. Поставте кількість раундів в підборі гіперпараметрів рівну **10**.

  **Увага!** Для того, аби скористатись hyperopt, нам треба задати функцію `objective`. І тут ми також ставимо loss - негативне значення AUROC, як і при пошуці гіперпараметрів для XGBoost. До речі, можна спробувати написати код так, аби в objective передавати лише модель і не писати схожий код двічі :)

  Після успішного завершення пошуку оптимальних гіперпараметрів
    - виведіть найкращі значення гіперпараметрів
    - створіть в окремій зміній `final_lgb_clf` модель `LightGBM` з найкращими гіперпараметрами
    - навчіть модель `final_lgb_clf`
    - оцініть якість моделі `final_lgb_clf` на тренувальній і валідаційній вибірках з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи стала вона краще порівняно з попереднім пунктом (4) цього завдання?

In [17]:
raw_df = pd.read_csv('../../../../bank-customer-churn-prediction-dlu/data/train.csv')

# Drop index column
raw_df.drop('id', axis=1, inplace=True)

# Drop 'Surname' column
raw_df.drop('Surname', axis=1, inplace=True)

train_df, val_df = split_train_val(raw_df, target_col='Exited')

train_inputs, train_targets = separate_inputs_targets(train_df, input_cols=raw_df.columns.drop('Exited').to_list(), target_col='Exited')
val_inputs, val_targets = separate_inputs_targets(val_df, input_cols=raw_df.columns.drop('Exited').to_list(), target_col='Exited')

for col in train_inputs.select_dtypes(include='object').columns:
    train_inputs[col] = pd.Categorical(train_inputs[col])
    val_inputs[col] = pd.Categorical(val_inputs[col])

cat_feature_indexes = [train_inputs.columns.get_loc(col) for col in train_inputs.select_dtypes(include='category').columns]

def objective(params):
    _lgb_clf = LGBMClassifier(
        n_estimators=int(params['n_estimators']),  # Кількість дерев у ансамблі (кількість ітерацій бустингу)
        learning_rate=params['learning_rate'],  # Коефіцієнт, на який зменшується внесок кожного доданого дерева
        max_depth=int(params['max_depth']),  # Максимальна глибина кожного дерева
        num_leaves=int(params['num_leaves']),  # Максимальна кількість листків, що дозволяємо кожному дереву мати.
        min_child_weight=params['min_child_weight'],  # Мінімальна сума ваг всіх вибірок, необхідна в кінцевому вузлі
        subsample=params['subsample'],  # Частка вибірок, що використовуються для побудови кожного дерева
        colsample_bytree=params['colsample_bytree'],  # Частка ознак, що використовуються при побудові кожного дерева
        reg_alpha=params['reg_alpha'],  # Параметр регуляризації L1 (Lasso)
        reg_lambda=params['reg_lambda'],  # Параметр регуляризації L2 (Ridge)
        min_split_gain=params['min_split_gain'],  # Мінімальне зменшення втрат, необхідне для виконання поділу
        early_stopping_rounds=10,
        n_jobs=-1,
        verbose=0,
    )

    _lgb_clf.fit(train_inputs, train_targets, categorical_feature=cat_feature_indexes, eval_set=[(val_inputs, val_targets)])
    val_auc = roc_auc_score(val_targets, _lgb_clf.predict_proba(val_inputs)[:, 1])
    loss = -val_auc  # negative AUROC
    return {'loss': loss, 'status': STATUS_OK}

space = {
    'n_estimators': hp.quniform('n_estimators', 50, 500, 25),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.3),
    'max_depth': hp.quniform('max_depth', 3, 15, 1),
    'num_leaves': hp.quniform('num_leaves', 20, 150, 1),
    'min_child_weight': hp.quniform('min_child_weight', 1, 10, 1),
    'subsample': hp.uniform('subsample', 0.5, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0),
    'reg_alpha': hp.uniform('reg_alpha', 0, 1),
    'reg_lambda': hp.uniform('reg_lambda', 0, 1),
    'min_split_gain': hp.uniform('min_split_gain', 0, 0.1)
}

trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=20,
            trials=trials)

best['n_estimators'] = int(best['n_estimators'])
best['max_depth'] = int(best['max_depth'])
best['num_leaves'] = int(best['num_leaves'])
best['min_child_weight'] = int(best['min_child_weight'])

print(best)

[LightGBM] [Warning] early_stopping_round is set=10, early_stopping_rounds=10 will be ignored. Current value: early_stopping_round=10
[LightGBM] [Warning] early_stopping_round is set=10, early_stopping_rounds=10 will be ignored. Current value: early_stopping_round=10
[LightGBM] [Warning] early_stopping_round is set=10, early_stopping_rounds=10 will be ignored. Current value: early_stopping_round=10
[LightGBM] [Warning] early_stopping_round is set=10, early_stopping_rounds=10 will be ignored. Current value: early_stopping_round=10
[LightGBM] [Warning] early_stopping_round is set=10, early_stopping_rounds=10 will be ignored. Current value: early_stopping_round=10
[LightGBM] [Warning] early_stopping_round is set=10, early_stopping_rounds=10 will be ignored. Current value: early_stopping_round=10
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf       
[LightGBM] [Warning] early_stopping_round is set=10, early_stopping_rounds=10 will be ignored. Current value: earl

In [22]:
final_lgb_clf = LGBMClassifier(
    n_estimators=best['n_estimators'],
    learning_rate=best['learning_rate'],
    max_depth=best['max_depth'],
    num_leaves=best['num_leaves'],
    min_child_weight=best['min_child_weight'],
    subsample=best['subsample'],
    colsample_bytree=best['colsample_bytree'],
    reg_alpha=best['reg_alpha'],
    reg_lambda=best['reg_lambda'],
    min_split_gain=best['min_split_gain'],
    verbose=0
)

final_lgb_clf.fit(train_inputs, train_targets, categorical_feature=cat_feature_indexes)


train_auc = roc_auc_score(train_targets, final_lgb_clf.predict_proba(train_inputs)[:, 1])
val_auc = roc_auc_score(val_targets, final_lgb_clf.predict_proba(val_inputs)[:, 1])

print(f"Train AUROC: {train_auc:.4f}")
print(f"Validation AUROC: {val_auc:.4f}")

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

**Висновок:**
Для LightGBM моделі я прибрав Surname колонку, щоб не було багато фіч для процесінгу. Це в свою чергу дало покращення значень auroc для тренувальних та валідаційних вибірках. На данний момент це найкраща модель. Але ми не пробували для інших моделей прибрати Surname

6. Оберіть модель з експериментів в цьому ДЗ і зробіть новий `submission` на Kaggle та додайте код для цього і скріншот скора на публічному лідерборді.
  
  **Напишіть коментар, чому ви обрали саме цю модель?**

  І я вас вітаю - це останнє завдання з цим набором даних 💪 На цьому етапі корисно проаналізувати, які моделі показали себе найкраще і подумати, чому.

In [24]:
# Read test data
test_df = pd.read_csv('../../../../bank-customer-churn-prediction-dlu/data/test.csv')
test_df_ids = test_df["id"].values

# Drop index column
test_df.drop('id', axis=1, inplace=True)

# Drop 'Surname' column
test_df.drop('Surname', axis=1, inplace=True)

# Convert categorical columns to 'category' type
for col in test_df.select_dtypes(include='object').columns:
    test_df[col] = pd.Categorical(test_df[col])

# Train final model
final_lgb_clf.fit(train_inputs, train_targets, categorical_feature=cat_feature_indexes)

# Make predictions
test_probas = final_lgb_clf.predict_proba(test_df)[:, 1]

# Create a submission
sample_raw_df = pd.DataFrame({'id': test_df_ids})
sample_raw_df['Exited'] = test_probas
sample_raw_df.to_csv("submission_final_lgb_clf.csv", index=False)

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f